In [48]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\toros\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [49]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1200
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = ['2020-01-01','2021-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01


In [50]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,start_year,end_year, Wanted_return, maximum_risk,'monthly') #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
sp500 = data.data_for_beta(np.append(['SPY'],(weights_of_each_portfolio.columns)),dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio)
capm_expected_return = capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)


c:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:40: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:113: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 0.000% => Return: 0.972%  Sharpe Ratio = 2307.28
Max. Sharpe Ratio = 200188.03 => Return: 16.57%  Risk: 0.000%
Excpected return on investment is 1.000000239235336%


In [51]:
weights_of_each_portfolio




,AWK,ANSS,APTV,ADM,AMZN,ALLE,AKAM,APD,ACN,ABT,...,TYL,URI,VRSN,VRSK,V,WM,WEC,WST,WAB,WY
0,0.072224,0.080939,0.016686,0.034663,0.175006,-0.046285,0.058019,-0.120699,-0.000892,0.038569,...,0.137353,0.257665,-0.193661,-0.005452,-0.107495,-0.10911,-0.137611,0.063681,-0.220931,-0.319255


In [52]:
portfolio_allocations

[120788.228036469]

In [53]:
portfolio_esg_score

,ESG_score_of_portfolio
0,1223.050304


In [54]:
cmle_returns

[18810.931131017016]

In [55]:
betas_of_portfolio

3.62235106771136e-07

In [56]:
capm_expected_return

0.01000000239235336

In [57]:
plot.plot_efficient_frontier(parameters,2011,2022,risk_free_rate)

In [58]:

ipywidgets.interactive(plot.plot_efficient_frontier_cml, parameters = ipywidgets.fixed(parameters),
                       start_year = ipywidgets.fixed(2011),
                       end_year = ipywidgets.fixed(2022),
                       risk_free_rate=ipywidgets.fixed(risk_free_rate),
                       plot_cml = True,
                       risk = (0,1,0.01),
                       mpl_style='default',
                       title='Efficient Frontier')

interactive(children=(Checkbox(value=True, description='plot_cml'), FloatSlider(value=0.1, description='risk',…